<a href="https://colab.research.google.com/github/naufalmufidf/TextSimilarityMeasure-JaccardSimilarity/blob/main/Jaccard%20Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import libs

In [1]:
!pip install pypdf2
!pip install python-docx
!pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 222 kB 4.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 5.0 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=19089f379f0bb77712035987a98869a25455a25eb5b522d493224d668789c425
  Stored in directory: /root/.cache/pip/wheels/32/b8/b2/c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 5.0 MB/s 


In [2]:
import numpy as np
import pandas as pd
import sys
import PyPDF2 as pypd
import docx
import re
import string
import collections
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
np.set_printoptions(threshold=sys.maxsize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


##Functions

In [3]:
def pdfview(path):
  openTxt = open(path, 'rb')
  readPDFTxt = pypd.PdfFileReader(openTxt)
  pages = readPDFTxt.numPages
  output = []
  for i in range(pages):
    page = readPDFTxt.getPage(i)
    output.append(page.extractText())

  seperator = ','
  PDFtxt = seperator.join(output)

  return PDFtxt

In [4]:
def wordview(path):
  openTxt = open(path, 'rb')
  txt = docx.Document(openTxt)
  Wordtxt = []
  for i in txt.paragraphs:
      Wordtxt.append(i.text)
  # return '\n'.join(Wordtxt)
  return Wordtxt

In [5]:
def pdf(path):
  openTxt = open(path, 'rb')
  readPDFTxt = pypd.PdfFileReader(openTxt)
  pages = readPDFTxt.numPages
  output = []
  for i in range(pages):
    page = readPDFTxt.getPage(i)
    output.append(page.extractText())
  
  return output

In [6]:
def word(path):
  openTxt = open(path, 'rb')
  txt = docx.Document(openTxt)
  Wordtxt = []
  for i in txt.paragraphs:
      Wordtxt.append(i.text)

  return Wordtxt

In [7]:
def lowercase(dataset):
  return dataset.apply(lambda x: ''.join([w for w in x.lower()]))

In [8]:
def preprocess_remove(to_remove, data):
    r = re.findall(to_remove, data)
    for i in r:
        data = re.sub(i, '', data)
    return data

In [9]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

In [10]:
def sw(data):
  stop_words = stopwords.words('indonesian')

  new_text = []
  word_dic = []
  for text in data:
      new_text2 = []
      for word in text.split():
          if word.lower() not in stop_words:
              new_text2.append(word.lower())
              word_dic.append(word.lower())
      new_text2 = " ".join(new_text2)
      new_text.append(new_text2)
  
  return new_text

In [11]:
def itung(data):
  total = len(data)
  print("Jumlah kata= ", total)
  
  count = collections.Counter(data).most_common()
  for i in count:
    print("%s\t: %d"%(i[0],i[1]))

In [12]:
def retrieve(path):
  doc = path
  if doc.endswith('.pdf'):
    hasil = pdf(doc)
    df = pd.DataFrame()
    df['Original'] = hasil
    df['Lower cased'] = lowercase(df['Original'])
    df['Clean_text'] = df['Lower cased']
    df['Clean_text'] = df['Clean_text'].str.replace('[^a-zA-Z]', ' ')
    df['Clean_text'] = np.vectorize(preprocess_remove)('http[\w]*',df['Clean_text'])
    df['Clean_text'] = np.vectorize(preprocess_remove)('@[\w]*',df['Clean_text'])
    df['Clean_text'] = df['Clean_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
    df['Stemmed'] = df['Clean_text'].apply(stem_sentences)
    df['Final'] = sw(df['Stemmed'])
    temp = df['Final'].values.tolist()
    temp = str(temp)
    temp = temp.split()
    hasil = itung(temp)


  elif doc.endswith('.docx'):
    hasil = word(doc)
    df = pd.DataFrame()
    df['Original'] = hasil
    df['Lower cased'] = lowercase(df['Original'])
    df['Clean_text'] = df['Lower cased']
    df['Clean_text'] = df['Clean_text'].str.replace('[^a-zA-Z]', ' ')
    df['Clean_text'] = np.vectorize(preprocess_remove)('http[\w]*',df['Clean_text'])
    df['Clean_text'] = np.vectorize(preprocess_remove)('@[\w]*',df['Clean_text'])
    df['Clean_text'] = df['Clean_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
    df['Stemmed'] = df['Clean_text'].apply(stem_sentences)
    df['Final'] = sw(df['Stemmed'])
    temp = df['Final'].values.tolist()
    temp = str(temp)
    temp = temp.split()
    hasil = itung(temp)


  else: 
    hasil = 'Harap masukan dokumen .pdf atau .docx'
  
  return print(hasil)


In [13]:
def prep(path):
  doc = path
  if doc.endswith('.pdf'):
    hasil = pdf(doc)
    df = pd.DataFrame()
    df['Original'] = hasil
    df['Lower cased'] = lowercase(df['Original'])
    df['Clean_text'] = df['Lower cased']
    df['Clean_text'] = df['Clean_text'].str.replace('[^a-zA-Z]', ' ')
    df['Clean_text'] = np.vectorize(preprocess_remove)('http[\w]*',df['Clean_text'])
    df['Clean_text'] = np.vectorize(preprocess_remove)('@[\w]*',df['Clean_text'])
    df['Clean_text'] = df['Clean_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
    df['Stemmed'] = df['Clean_text'].apply(stem_sentences)
    df['Final'] = sw(df['Stemmed'])
    temp = df['Final'].values.tolist()
    temp = str(temp)
    temp = temp.split()
    # temp = ','.join(str(temp))
    hasil = temp
  
  elif doc.endswith('.docx'):
    hasil = word(doc)
    df = pd.DataFrame()
    df['Original'] = hasil
    df['Lower cased'] = lowercase(df['Original'])
    df['Clean_text'] = df['Lower cased']
    df['Clean_text'] = df['Clean_text'].str.replace('[^a-zA-Z]', ' ')
    df['Clean_text'] = np.vectorize(preprocess_remove)('http[\w]*',df['Clean_text'])
    df['Clean_text'] = np.vectorize(preprocess_remove)('@[\w]*',df['Clean_text'])
    df['Clean_text'] = df['Clean_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
    df['Stemmed'] = df['Clean_text'].apply(stem_sentences)
    df['Final'] = sw(df['Stemmed'])
    temp = df['Final'].values.tolist()
    temp = str(temp)
    temp = temp.split()
    # temp = ','.join(str(temp))
    hasil = temp
  
  else: 
    hasil = 'Harap masukan dokumen .pdf atau .docx'
  
  return hasil

##Main

###Count words

In [19]:
retrieve('/content/sample_data/PENERAPAN NEURAL NETWORK TENTANG METODE BACKPROPAGATION PADA PENGENALAN POLA HURUF aul2.docx')

<ipython-input-12-476a920b3fb6>:27: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Clean_text'] = df['Clean_text'].str.replace('[^a-zA-Z]', ' ')


Jumlah kata=  726
vektor	: 29
'',	: 24
input	: 24
huruf	: 23
output	: 22
latih	: 20
pola	: 19
lapis	: 19
noise	: 13
kenal	: 12
neuron	: 12
galat	: 11
hidden	: 11
metode	: 8
bobot	: 8
ideal	: 8
nali	: 7
sistem	: 7
bernoise	: 7
neural	: 6
network	: 6
proses	: 6
salah	: 5
coba	: 5
pasang	: 5
buah	: 5
fungsi	: 5
bobot',	: 5
alpabet	: 5
rangkap	: 5
klasifikasi	: 4
jalar	: 4
dimensi	: 4
target	: 4
ambang	: 4
aktivasi	: 4
hitung	: 4
'latih	: 4
wakil	: 4
backpropagation	: 3
prediksi	: 3
software	: 3
batas	: 3
bidang	: 3
analisis	: 3
suara	: 3
ukur	: 3
ulang	: 3
ajar	: 3
pilih	: 3
'bobot	: 3
unit	: 3
iterasi	: 3
nilai	: 3
merespon	: 3
uji	: 3
huruf',	: 2
teknologi	: 2
aplikasi	: 2
simulasi	: 2
paper	: 2
noise',	: 2
baca	: 2
orang	: 2
mudah	: 2
tebak	: 2
manusia	: 2
komputer	: 2
pesawat	: 2
terbang	: 2
kontrol	: 2
deteksi	: 2
manufaktur	: 2
'metode	: 2
backpropagation',	: 2
hubung	: 2
rupa	: 2
capai	: 2
kali	: 2
meta	: 2
ruang	: 2
input',	: 2
muncul	: 2
bias	: 2
koneksi	: 2
dasar	: 2
jalar',	: 2

In [20]:
retrieve('/content/sample_data/Project Capstone Kelompok 11_Dzakiyyah_Fayza_Naufal_AI Hacker (2).pdf')

<ipython-input-12-476a920b3fb6>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Clean_text'] = df['Clean_text'].str.replace('[^a-zA-Z]', ' ')


Jumlah kata=  2119
data	: 93
gambar	: 49
harga	: 43
model	: 38
metode	: 38
dataset	: 38
jual	: 34
produk	: 32
hasil	: 32
random	: 31
forest	: 31
laku	: 31
prediksi	: 27
algoritma	: 27
visualisasi	: 25
teliti	: 23
shopee	: 19
dasar	: 17
price	: 16
online	: 15
score	: 14
salah	: 14
brand	: 13
tuju	: 13
tahap	: 13
tree	: 12
kolom	: 12
rupa	: 11
regression	: 11
tuning	: 11
banding	: 10
mape	: 10
regresi	: 10
hyperparameter	: 10
market	: 10
count	: 10
analytics	: 9
pasar	: 9
variabel	: 9
sample	: 9
rumus	: 9
basket	: 9
proses	: 9
value	: 9
label	: 9
'gambar	: 9
commerce	: 8
decision	: 8
lasso	: 8
informasi	: 8
milik	: 8
ambil	: 8
pengaruh	: 8
feature	: 8
kategori	: 8
selection	: 8
missing	: 8
scraping	: 8
analisis	: 7
linear	: 7
indonesia	: 7
pinjam	: 7
processing	: 7
tingkat	: 7
encoder	: 7
category	: 7
search	: 7
hadap	: 6
belanja	: 6
website	: 6
luas	: 6
neighbors	: 6
nilai	: 6
putus	: 6
marketplace	: 6
akurasi	: 6
terap	: 6
klasifikasi	: 6
index	: 6
handling	: 6
available	: 6
teknologi	

In [21]:
retrieve('/content/sample_data/Penerapan Machine Learning terhadap Klasifikasi Kanker Payudara pada Pasien (2).docx')

<ipython-input-12-476a920b3fb6>:27: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Clean_text'] = df['Clean_text'].str.replace('[^a-zA-Z]', ' ')


Jumlah kata=  1296
kanker	: 39
data	: 37
payudara	: 35
'',	: 30
mean	: 23
worst	: 22
algoritma	: 21
learning	: 18
hasil	: 18
klasifikasi	: 17
kolom	: 16
laku	: 15
random	: 15
score	: 15
model	: 15
forest	: 14
'laku	: 14
machine	: 13
proses	: 13
tree	: 11
'gambar	: 11
rupa	: 10
jenis	: 10
banding	: 10
diagnosis	: 10
tuning	: 10
milik	: 9
accuracy	: 9
dataset	: 9
radius	: 9
perimeter	: 9
coba	: 8
decision	: 8
area	: 8
kembang	: 7
metode	: 7
classifier	: 7
compactness	: 7
concavity	: 7
concave	: 7
points	: 7
program	: 6
vector	: 6
missing	: 6
feature	: 6
hyperparameter	: 6
hadap	: 5
sistem	: 5
informasi	: 5
teknologi	: 5
dasar	: 5
sehat	: 5
pasien	: 5
jaring	: 5
supervised	: 5
prediksi	: 5
value',	: 5
informatika	: 4
ajar	: 4
tingkat	: 4
ganas	: 4
mati	: 4
negara	: 4
deteksi	: 4
pengaruh	: 4
bidang	: 4
support	: 4
wanita	: 4
derita	: 4
label	: 4
target	: 4
nilai	: 4
'tahap	: 4
dataset',	: 4
'dataset	: 4
beda	: 4
rescale	: 4
kece	: 4
studi	: 3
artificial	: 3
alam	: 3
akibat	: 3
sakit	: 3
p

In [22]:
retrieve('/content/sample_data/Kelompok7_KmeansClustering.pdf')

<ipython-input-12-476a920b3fb6>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Clean_text'] = df['Clean_text'].str.replace('[^a-zA-Z]', ' ')


Jumlah kata=  1517
cluster	: 108
data	: 80
kelompok	: 42
pusat	: 36
clustering	: 32
badan	: 24
berat	: 24
jarak	: 23
means	: 18
milik	: 18
hitung	: 18
metode	: 15
mahasiswa	: 15
algoritma	: 11
dasar	: 11
anggota	: 11
label	: 11
kmeans	: 10
wcss	: 9
studi	: 8
output	: 8
proses	: 8
lihat	: 8
mining	: 7
teknologi	: 7
objek	: 7
tuju	: 6
kerja	: 6
informasi	: 6
tingkat	: 6
dokumen	: 6
cari	: 6
teknik	: 6
centroid	: 6
hadap	: 6
kenal	: 6
rupa	: 6
point	: 6
nilai	: 6
lengkap	: 6
ubah	: 6
import	: 6
scatter	: 6
plot	: 6
sejarah	: 5
erti	: 5
tugas	: 5
beda	: 5
titik	: 5
arti	: 5
hasil	: 5
nama	: 5
tabel	: 5
pilih	: 5
clusters	: 5
warna	: 5
saran	: 4
makalah	: 4
gambar	: 4
terima	: 4
laku	: 4
buah	: 4
kumpul	: 4
butuh	: 4
analisis	: 4
temu	: 4
machine	: 4
learning	: 4
ajar	: 4
pola	: 4
cluster',	: 4
pendek	: 4
maulana	: 4
rizky	: 4
saputra	: 4
masuk	: 4
tampil	: 4
dataframe	: 4
null	: 4
init	: 4
elbow	: 4
terang	: 4
tanda	: 4
murid	: 4
jasman	: 3
program	: 3
rumus	: 3
implementasi	: 3
simpul	: 3

In [23]:
retrieve('/content/sample_data/Modul K-Means.pdf')

<ipython-input-12-476a920b3fb6>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Clean_text'] = df['Clean_text'].str.replace('[^a-zA-Z]', ' ')


Jumlah kata=  949
centroid	: 30
data	: 29
klaster	: 25
iterasi	: 23
hitung	: 21
berat	: 16
cluster	: 13
jarak	: 12
rizky	: 12
saputra	: 12
badan	: 11
maulana	: 11
clustering	: 9
firmansyah	: 9
means	: 8
noval	: 8
akhmad	: 8
husni	: 8
hasil	: 8
dataset	: 7
posisi	: 7
mahasiswa	: 7
nurlaela	: 7
musnadil	: 7
firdaus	: 7
fauzan	: 7
abdullah	: 7
faqih	: 7
muhammad	: 7
rizki	: 7
bagus	: 7
setiawan	: 7
fikriyah	: 7
khairunnisa	: 7
irvan	: 7
akbar	: 7
febriansyah	: 7
millati	: 7
rachma	: 7
rizqi	: 7
fauzi	: 7
scatter	: 7
nilai	: 6
nama	: 6
ikhlasul	: 6
amal	: 6
awaludin	: 6
aida	: 6
bani	: 6
arief	: 6
jasmine	: 6
insan	: 6
indah	: 6
sari	: 6
efendi	: 6
kisnoto	: 6
muhamad	: 6
ilham	: 6
fajar	: 6
sidiq	: 6
bella	: 6
fitriani	: 6
putri	: 6
nafianto	: 6
faizal	: 6
wibowo	: 6
meyfandi	: 6
ardaniawan	: 6
doni	: 6
cahya	: 6
farid	: 6
nurul	: 6
ikhsani	: 6
ana	: 6
zuhudistia	: 6
elang	: 6
bimantoro	: 6
abbror	: 6
maksum	: 6
rupa	: 5
kelompok	: 5
pusat	: 5
pilih	: 5
rumus	: 5
selesai	: 5
ariffulah	: 5

###Measure

In [25]:
x = prep('/content/sample_data/PENERAPAN NEURAL NETWORK TENTANG METODE BACKPROPAGATION PADA PENGENALAN POLA HURUF aul2.docx')
y = prep('/content/sample_data/Penerapan Machine Learning terhadap Klasifikasi Kanker Payudara pada Pasien (2).docx')
z = prep('/content/sample_data/Project Capstone Kelompok 11_Dzakiyyah_Fayza_Naufal_AI Hacker (2).pdf')
h = prep('/content/sample_data/Kelompok7_KmeansClustering.pdf')
i = prep('/content/sample_data/Modul K-Means.pdf')

<ipython-input-13-0c0e805c58f8>:27: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Clean_text'] = df['Clean_text'].str.replace('[^a-zA-Z]', ' ')
<ipython-input-13-0c0e805c58f8>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Clean_text'] = df['Clean_text'].str.replace('[^a-zA-Z]', ' ')


In [26]:
def measure(a, b):
  doc1 = set(a)
  doc2 = set(b)
  intersection= doc1.intersection(doc2)
  union= doc1.union(doc2)

  return float(len(intersection))/len(union)
  # return len(union)

In [27]:
measure(x,y)

0.09469153515064563

In [28]:
measure(x,z)

0.08139534883720931

In [29]:
measure(y,z)

0.17753259779338015

In [30]:
measure(h,i)

0.2635024549918167